In [ ]:
import pandas as pd
import tweepy

## Sign-up & Authentification

In [ ]:
api_key = api_key
api_key_secret = api_key_secret

access_token = access_token
access_token_secret = access_token_secret

In [ ]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

In [ ]:
api = tweepy.API(auth)

In [ ]:
dir(api)

In [ ]:
api.update_status('Twitter API trial')

In [ ]:
api.destroy_status(id = 1502097617140203520)

## Json

Official doc: https://www.json.org/json-en.html

- A data format designed for data exchange between systems and languages

- Essentially a tree-like structure built on two types of structure common in most programming languages (akin to dictionary and list in Python)

## Put tweets into a DataFrame

In [ ]:
my_timeline = api.home_timeline()

In [ ]:
my_timeline

In [ ]:
tweet_5 = my_timeline[4]
tweet_5

In [ ]:
tweet_5.text

In [ ]:
tweet_5.user.screen_name

In [ ]:
vars(tweet_5)

In [ ]:
vars(tweet_5)['text']

In [ ]:
vars(tweet_5)['user'].screen_name

In [ ]:
headers = set()
# Python lists allow duplicate elements; sets do not.
tweets_data = []

for tweet in my_timeline:
    tweet_dict = vars(tweet)
    keys = tweet_dict.keys()
    single_tweet_data = {"user": tweet.user.screen_name}
    for k in keys:
        v_type = type(tweet_dict[k])
        if v_type in [str, int]:
            single_tweet_data[k] = tweet_dict[k]
            headers.add(k)
    tweets_data.append(single_tweet_data)
    
columns = list(headers)
columns.append("user")

In [ ]:
columns

In [ ]:
tweets_data

In [ ]:
df = pd.DataFrame(tweets_data, columns=columns)
df.head()

In [ ]:
def extract_timeline_df(timeline):
    headers = set()
    tweets_data = []

    for tweet in timeline:
        tweet_dict = vars(tweet)
        keys = tweet_dict.keys()
        single_tweet_data = {"user": tweet.user.screen_name}
        for k in keys:
            v_type = type(tweet_dict[k])
            if v_type in [str, int]:
                single_tweet_data[k] = tweet_dict[k]
                headers.add(k)
        tweets_data.append(single_tweet_data)

    columns = list(headers)
    columns.append("user")
    
    df = pd.DataFrame(tweets_data, columns = columns)
    return df

## Tweets from other users and hashtags

In [ ]:
# Let's check the latest tweets of the author of our textbook — Matthew Salganik

user = 'msalganik'
user_timeline = api.user_timeline(screen_name = user, count = 10)

df_2 = extract_timeline_df(user_timeline)
df_2

In [ ]:
df_2['text'][6]

In [ ]:
user_timeline = api.user_timeline(screen_name = user, count = 300)

df_3 = extract_timeline_df(user_timeline)
df_3

There's a maximum number of tweets `user_timeline` can extract. To extract more tweets, we use `Cursor`

In [ ]:
user_timeline = tweepy.Cursor(api.user_timeline, screen_name = user).items(300)
df_3 = extract_timeline_df(user_timeline)
df_3

In [ ]:
# search tweets by a keyword

keywords = ['China', '#EconTwitter']
search_timeline = tweepy.Cursor(api.search_tweets, q = keywords).items(10)

df_4 = extract_timeline_df(search_timeline)
df_4

## Streaming

In [ ]:
# create a subclass of tweepy.Stream

class Linstener(tweepy.Stream):

    tweets = []
    limit = 30
    

    def on_status(self, status):
        self.tweets.append(status)
        
        # print(status.user.screen_name + ": " + status.text)
        
        # stop streaming after 30 tweets
        if len(self.tweets) == self.limit:
            self.disconnect()

# Notice Stream is a different application from API, we need to authenticate again. 
stream_tweet = Linstener(api_key, api_key_secret, access_token, access_token_secret)

In [ ]:
# stream by keywords

keywords = ['Ukraine']

stream_tweet.filter(track=keywords)

In [ ]:
# Put streaming tweets into df

columns = ['User', 'Tweet']
data = []

for tweet in stream_tweet.tweets:
    data.append([tweet.user.screen_name, tweet.text])

df = pd.DataFrame(data, columns=columns)
df

For more on Twitter Streaming, see https://developer.twitter.com/en/docs/tutorials/consuming-streaming-data